In [2]:
import sys
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
# %matplotlib inline

In [3]:
import numpy
from scipy.stats import norm

def normalised_implied_volatility(x, v):
     
    shape = (x*v).shape
    
    converged = numpy.full(shape, False, dtype='bool')
    results = numpy.full(shape, numpy.nan)
    
    i = 0
    
    mask = 0.0 == v
    converged[mask] = True
    results[mask] = 0.0
    
    if converged.all():
        return results
        
    mask = numpy.logical_and(numpy.logical_not(converged), (1.0 == v))
    converged[mask] = True
    results[mask] = numpy.inf

    if converged.all():
        return results
    
    # only defined for +ve x (i.e. we should only be looking at OTM options)
    mask = x < 0.0
    converged[mask] = True
    results[mask] = numpy.nan

    # should have nothing below zero or above 1
    # options worth less than intrinsic or more that the underlying or strike
    mask = numpy.logical_or(v < 0.0, v > 1.0)
    converged[mask] = True
    results[mask] = numpy.nan
        
    y = numpy.where(converged, numpy.nan, 2.0 * norm.ppf((1.0 + v) / 2.0))
    
    mask = numpy.logical_and(numpy.logical_not(converged), y == numpy.inf)
    converged[mask] = True
    results[mask] = y[mask]
    if converged.all():
        return results
        
    mask = numpy.logical_and(numpy.logical_not(converged), 0 == x)
    converged[mask] = True
    results[mask] = y[mask]
    if converged.all():
        return results
         
    fOld = numpy.full(shape, numpy.inf)
    yOld = numpy.full(shape, numpy.inf)
    ex = numpy.where(converged, numpy.nan, numpy.exp(x))
    
    # testing over a range of values (different orders of magnitude over x and v)
    # a best worst case scenario seems to be two iterations of this fixed point approach
    # before reverting to newton/halley
    # (also, the "best worst-case" doesnt' much impact the 'iterations of the usual cases)
    for k in xrange(2):
        vEst = numpy.where(converged, numpy.nan, v + ex * norm.pdf(-x / y - y / 2.0))
        vEst[vEst >= 1.0] = numpy.broadcast_to(numpy.maximum(v,0.99), vEst.shape)[vEst >= 1.0]
        tmp = numpy.where(converged, numpy.nan, norm.ppf(vEst))
        tmp += numpy.sqrt(tmp * tmp + 2 * x)
        y[tmp != 0] = tmp[tmp != 0]
        ++i
    
    converged[0 == y] = True
    results[0 == y] = 0
    if converged.all():
        return results
         
    while i!=1000:
 
        if (y < 0.0).any():
            raise RuntimeError("oh dear, y should never go negative...")
        d1 = numpy.where(converged, numpy.nan, -x / y)
        y2 = numpy.where(converged, numpy.nan, y / 2.0)
        d2 = numpy.where(converged, numpy.nan, d1 - y2)
        d1 += y2
 
        p1 = numpy.where(converged, numpy.nan, norm.pdf(d1))
        p2 = numpy.where(converged, numpy.nan, norm.pdf(d2))
        r1 = numpy.where(converged, numpy.nan, norm.cdf(d1)/norm.pdf(d1)) # TODO maybe redundant: cdf is often implemented in terms of this ratio...
        r2 = numpy.where(converged, numpy.nan, norm.cdf(d2)/norm.pdf(d2))
        
        # if we're too far out & cdfToPdfRatio fails, rely on the fact that our guess is asymptotically correct
        mask = numpy.logical_and(numpy.logical_and(numpy.logical_not(converged), d2 < -30) ,numpy.isnan(r2))
        converged[mask] = True
        results[mask] = y[mask]
        if converged.all():
            return results 
 
        f = numpy.where(converged, numpy.nan, r1 * p1 - ex * r2 * p2 - v)
        mask = 0==f
        converged[mask] = True
        results[mask] = y[mask]
        if converged.all():
            return results
            
        mask = abs(fOld) <= abs(f)
        converged[mask] = True
        results[mask] = yOld[mask]
        if converged.all():
            return results
            
        yOld = numpy.where(converged, numpy.nan, y)
        fOld = numpy.where(converged, numpy.nan, f)
 
        dy = numpy.where(converged, numpy.nan, r1 - r2 - v / p1) # this would be a basic newton step
        dy = numpy.where(converged, numpy.nan, dy/(1.0 - dy * d1 * d2 / y)) # but we adjust, halley's method
        y = numpy.where(converged, numpy.nan, y -dy)
        i = i+1
 
    raise RuntimeError("Implied volatility failed to converge for inputs x = {}, v = {}".format(x,v))
    
def normalised_price(x, y):
    
    return numpy.where(numpy.logical_and(x <= 0.0, numpy.isinf(x)),1.0, # zero-strike call is worth 1 forward
                numpy.where(numpy.logical_and(x > 0.0, numpy.isinf(x)),0.0, # infinity-strike call is worthless
                                   numpy.where(y==0.0,numpy.maximum(1-numpy.exp(x), 0.0), # zero time call is intrinsic
                                               norm.cdf(-x/y+y/2)-numpy.exp(x)*norm.cdf(-x/y-y/2))))


The received wisdom states that implied Black-Scholes volatility has no closed form and must be implemented via Newton-Raphson or similar. The function signature for a typical implementation is something like:
``` python
def iv(spot, strike, expiry_time, rate, div_yield, opt_price, is_call, price_tol=1e-6, max_iter=500)
```

This note aims to simplify this situation somewhat, based on two observations
 - That signature is awfully long-winded. The implied volatiltiy function with its many inputs and all it varied generalisations, can always be reduced to a simple function of two real variables.
 - I don't have to specify ```max_iter``` or ```tol``` for other functions with no "closed form" such as ```math.log```, ```math.sqrt``` or ```norm.ppf```. Why in the age of Alpha Go Zero am i still worrying each time about how fast or how accurate ``iv`` has to be? Frankly, I'm both impatient **and** intolerant -  I 'd like the right answer, quickly, always, please.

# Why isn't implied volatility "closed form"?

It is usually stated that implied volatility (the inversion of the Black-Scholes option pricing formula with respect to the volatility argument) does not have a closed form solution and requires a numerical implementation, such as Newton Raphson. While this is technically true, this is not an argument that we accept for other fundamental mathematical functions (and implied volatility is certainly fundamental in the field of computational finance). We don't for example, expect to specify the precision tolerance or maximum number of iterations for the internal algorithm used in the computation of a square root or a natural logarithm - we demand an implementation that is fast and accurate over the full domain. It is often supposed that this is not practicable for implied volatility, perhaps because that function takes many arguments and is nonlinear in all of them.

It is common therefore - even considered "good practice" - that the implementation require the user to specify their tolerances for precision and computational effort, with little guidance about the appropriateness of those values or the tradeoff between speed and precision, which may vary over the domain (in strike ans expiry) and may differ depending on the use case.

In this note we discuss how the implied volatiltiy of any European option can always be implemented in terms of that of a 1-year out of the money call with unit underlying price and zero dividends or interest rates. Consequently the problem is reduced to implementation of a function of two arguments (option price, and strike level), whose behaviour is easily studied and understood over the whole domain. This enables us to develop an implementation that is stable, accurate and fast enough (even without rercourse to a low-level language like C++) to be used universally, without requiring user-specified tolerances. The resulting function of 2 arguments could arguably be considered an "elementary" function (as are the square root, logarithm etc.), in terms of which European option implied volatility can always be expressed in "closed form".

In [4]:
n=21
x_in = np.linspace(0,5,n)
y_in = np.linspace(0,2,n)
v_in = np.linspace(0,1,n)[1:]

x1, y1 = np.meshgrid(x_in, y_in)
v1 = normalised_price(x1, y1)

x2, v2 = np.meshgrid(x_in, v_in)
y2 = normalised_implied_volatility(x2, v2)

fig=plt.figure(figsize=(12,4))
ax= fig.add_subplot(1,2,1,projection='3d')
ax.plot_surface(x1, y1, v1, cmap=cm.coolwarm, rstride=1, cstride=1, linewidth=0)
ax.set_title("Option price v(x,y)")
ax.set_xlabel("x")
ax.set_ylabel("y")

ax= fig.add_subplot(1,2,2,projection='3d')
ax.plot_surface(x2, v2, y2, cmap=cm.coolwarm, rstride=1, cstride=1, linewidth=0);
ax.set_xlabel("x")
ax.set_ylabel("v")
ax.set_title("Implied volatility y(x,v)");

## Reduction to 2 dimensions

The price formulas for European call and put options are respectively

$$C=e^{-r(T-t)}(F\mathcal N(d_1)-K \mathcal N(d_2))$$

$$P=e^{-r(T-t)}(K\mathcal N(-d_2)- F\mathcal N(-d_1))$$

where

$$d_{1/2}=-\frac xy\pm\frac y2,\quad x=\ln\frac KF,\quad y=\sigma\sqrt{T-t}$$

Rearranging we can write

$$\frac{Ce^{r(T-t)}}{F}=\mathcal N(d_1)-e^x\mathcal N(d_2):=v(x,y)$$

$$\frac{Pe^{r(T-t)}}{K}=\mathcal N(-d_2)-e^{-x}\mathcal N(-d_1)=v(-x,y)$$

where in the latter case we have noted that $d_{1/2}(x,y)=-d_{2/1}(-x,y)$. So, both calls and puts can be defined in terms of $v(x,y)$, the undiscounted price of a call relative to the forward - or equivalently, assuming unit forward. Note that $v$ takes values in [0,1], since the call cannot be worth more than the forward (and the put cannot be worth more than the strike).

For the implied volatility  for calls and puts then we have

$$\sigma_{call}=y(x,\frac{Ce^{r(T-t)}}{F})/\sqrt{T-t}$$

$$\sigma_{put}=y(x,\frac{Pe^{r(T-t)}}{K})/\sqrt{T-t}$$

We prefer to only compute implied voaltiltiy for out-of-the-money options, since their near-zero values can be more accurately represented in floating point arithmetic leading to better accuracy. Therefore it is preferable to convert out-of-the money options into in-the-money ones via the put call parity relationship, $Ce^{r(T-t)}+Pe^{r(T-t)}=F-K$. That is, if we have an ITM call corresponding to $F>K$ i.e. $x<0$, we can say, since the implied volatlity of the call and put must equate,

$y(x,\frac{Ce^{r(T-t)}}{F})=y(-x,\frac{F-K-Ce^{r(T-t)}}{K})$

Similarly, for an ITM put option with $F<K$ and thus $x>0$ we may rewrite the implied volatility as

$y(-x,\frac{Pe^{r(T-t)}}{K})=y(x,\frac{F-K-Pe^{r(T-t)}}{F})$

In both cases, we are making use of the put-call symmetry property of $y$, that

$y(x,v)=y(-x,e^{-x}-v-1)$

meaning that we need only evaluate y for $x>0$, which can be done with greater numerical accuracy, as $v\rightarrow 0$, than for $x<0$ where $v\rightarrow 1$.

## Implementation

#### Initial Guess

Since we have a closed form solution $y$ in the case of at-the-money options $x=0$, we take our initial guess for $y(x,v)$ to be $y(0,v)$.

$$v(0,y)=\mathcal N (y/2)- \mathcal N(-y/2)=2\mathcal N(y/2)+1\Rightarrow y(0,v)=2\mathcal N^{-1}((1+v)/2))$$

Obviously no further computation is needed in the case that we are computing an at-the-money implied volatility.

#### Implicit formula iteration

Our initial guess is good for close to at-the-money options but may be innacurate for way out of the money options. Rather than rely immediately on a generic Newton iteration, we can use our knowledge of the function $v$ to quickly find a better starting approximation to such a generic method. The following iterative procedure improves our guess for all values of $x$, but importantly is asymptotically exact for large x and so improves very quickly for out of the money options.

We are motivated by the fact that in our function $v=\mathcal N(d_1)-e^x\mathcal N(d-2)$, it is the first term that dominates for large x, and we find $v(x,y)\rightarrow\mathcal N(d_1)$. We can then rearrange the formula as

$d_1=\mathcal N^{-1}(v+e^x\mathcal N(d_2))$

This is a quadratic in $y$, with the solution

$y=\sqrt{\mathcal N^{-1}((v+e^x\mathcal N(d_2)))^2+2x}$

This formula is implicit as $y$ appears in the $d_2$ on the right hand side. However, the effect of the $e^x\mathcal N(d_2)$ becomes negligible, because of the aforementioned large-$x$ behaviour of $v(x,y)$. Therefore an iterative procedure using this implicit formulation is very accurate for larger $x$. This is particularly useful because we know that the gradient, i.e. the option vega, vanishes for large $x$ and so a generic Newton approach becomes ineffective. But it is in precisely this case that this alternative method is near-exact, with accuracy constrained only by the implementation of the inverse normal $\mathcal N^{-1}$. It is this fact that allows us stable and accurate inversion over extreme strike ranges, right down to the limits of floating point arithmetic, as we will see in our later results.

Informed by empirical tests of the efficacy of this approach across the range of possible inputs, our implementation always takes 2 steps of this approach before reverting to the generic iterative procedure.

#### Newton-Halley iteration

The standard Newton-Raphson approach based on the gradient ("vega") can be augmented by retaining the 2nd derivative ("volga") in the Taylor expansion to give cubic rather than quadratic rate of convergence - this is 'Halley's method'. In the case of our function, because the volga/vega ratio simplifies neatly, this results in a quite simpe correction. The standard Newton step would be to adjust our $y$ estimate by

$\delta y_N = \frac{\mathcal N(d_1)}{\mathcal N'(d_1)}-\frac{\mathcal N(d_2)}{\mathcal N'(d_2)}-\frac{v}{\mathcal N'(d_2)}$

The Halley step adjusts this to

$$\delta y_H = \frac{\delta y_N}{1-\delta y_N \frac{d_1d_2}{y}}$$



#### Convergence criterion

As mentioned in the introduction, we do not consider it desirable that the user is requried to specify a tolerance level. Since we know that our function is sufficiently well behaved that every Newton-Halley step improves the result, we stop when the limits of floating point arithmetic cause this to be contradicted, i.e. when $\vert v(x,y_i)-v\vert >= \vert v(x,y_{i-1})-v\vert$. That is, we keep going until we literally can't improve any more. We do not consider this to be 'overkill', being of the opinion that it is best to have the highest level of accuracy available, subject to considerations of sufficient computational speed which, as we will see in our tests, are happily satisfied.

#### Vectorisation

Our implementation accepts numpy arrays as inputs and broadcasts to return results of the approproate dimension. Internally, computations are using numpy and so are efficiently vectorised. The iterative procedure repeats until all calcualtions have converged - however the internal calculations at each step are only made for the points which arenot yet converged, thereby increasing efficiency. This means that the only "for" loop is the outer loop over iterations - all calls to the normal distribution functions norm.ppf and norm.cdf are efficiently vectorised for example.

## Results

In this section we demonstrate the accuracy of our implementation. We look at the error in volatiltiy space, and in price space both in relative and absolute terms.

The option value function that we wish to invert is shown for options of various maturity in the following figure. We consider options of various maturities from 1 second to 1 year with the volatility standardised at 0.1 (without loss of generality since the price depends on volatility only through the total standard deviation $\sigma\sqrt{T-t}$). For visualisation purposeswWe rescale relative to the ATM option of the same maturity so that all appear on the same vertical scale, i.e. we plot v(x,y)/v(0,y). On the horizontal axis we plot not against log-moneyness $x$, but against x/y, i.e. the number of standard deviations from ATM. Normalised in both directions in this way the plot lines for differing maturities appear on the same scale (indeed they appear visually indistinguishable, though they are numerically different)

We consider out to 37 standard deviations on the horizontal axis. For "1 second to expiry" options at 10% volatiltiy this corresponds to 6bp in/out of the money. For 1 year options at the same volatility, it is (for a \$1 underlying), a \$33-strike call and a \$0.03-strike put. These are the ranges over which we will test the accuracy of our inversion process.

In [5]:
npoints = 10000 # how many points on each plot
sdmax = 35 # how many standard deviations do we consider on the horizontal axis
sd=np.linspace(0,sdmax,npoints).reshape(1,npoints)

vol = 0.1
t = np.array([1./365./24./60./60., 1./365./24./60., 1./365./24., 1./365., 7./365., 1./12., 1.])
t_labels = np.array(["1s","1m","1h","1d","1w","1m","1y"])
y = (vol * np.sqrt(t)).reshape(len(t), 1)

x = y * sd
v=normalised_price(x,y)

data = pd.DataFrame(data=v, index=t_labels, columns=sd.flatten())
#data.div(data[0.0], axis=0).transpose().plot(figsize=(12,4), logy=True);

data.transpose().plot(figsize=(12,4), logy=False);
data.transpose().plot(figsize=(12,4), logy=True);


### Computational speed

We invert the option price function at each of the specified values. The computational speed of this operation is summarised below.

In [6]:
import time
cl = time.clock()
y_test = normalised_implied_volatility(x,v) 
cl = time.clock()-cl
vol_test = y_test / np.sqrt(t).reshape(len(t),1)
n_points = int(y_test.shape[0] * y_test.shape[1])

df = pd.DataFrame(pd.Series({'Function Calls' : n_points, 'Total time (ms)' : 1000 * cl, 'Average time per call (ms)' : 1000 * cl/n_points}))
df.transpose()

,Average time per call (ms),Function Calls,Total time (ms)
0,0.034706,70000.0,2429.396484


### Precision in volatility space

When we compute the implied volatilty using our implementation for these same values, we find errors of the order 10^-12. These are largest for short-date options, and are constant in order of magnitude across the strike range.

In [7]:
pd.DataFrame(data = vol_test - 0.1, index=t_labels, columns=sd.flatten()).transpose().plot(figsize=(12,4), style=",", ms=3)
pd.DataFrame(data = np.abs(vol_test - 0.1), index=t_labels, columns=sd.flatten()).transpose().plot(figsize=(12,4), logy=True, style=",", ms=3)

### Precision in price space

In price space, the absolute error (difference in price between v(x,y(x,v(x,y)))-v(x,y) is of the order $10^-16$, across all maturities, and is largest at the money, reducing quickly by orders of magnitude as x increases.

The relative price error, v(x,y(x,v(x,y)))/v(x,y) -1, is of the order $10^-9$, increasing in magnitude with x.

In [8]:
v_err=normalised_price(x,y_test)-v

fig, ax = plt.subplots(2,2,figsize=(12,6))

pd.DataFrame(data=v_err, index=t_labels, columns=sd.flatten()).transpose().plot(style="+", ms=1, ax=ax[0,0], title="$err")
pd.DataFrame(data=np.abs(v_err), index=t_labels, columns=sd.flatten()).transpose().plot(style="+", ms=1, ax=ax[0,1], logy=True, title="|$err| logscale")
pd.DataFrame(data=v_err/v, index=t_labels, columns=sd.flatten()).transpose().plot(style="+", ms=1, ax=ax[1,0], title="%err")
pd.DataFrame(data=np.abs(v_err/v), index=t_labels, columns=sd.flatten()).transpose().plot(style="+", ms=1, ax=ax[1,1], logy=True, title="|%err| logscale");

### Limiting behaviour at large $x$

It is worth remarking that by the time we get to 37 standard deviations, the extreme limit of these tests, our option prices (across all maturites) are smaller than 10^-300. The smallest positive Python float number distinguishable from zero (found using "numpy.nextafter") is around 10^-324. So by the point our implied volatiltiy computation fails around 38 or 39 standard deviations (not shown), we are really not all that far from the hard theoretical limit of the option price itself being effectively precisely 0.0. In fact, since we know that for large $x$ (extreme out-of-the-money options) our initial guess is asymptotically correct, it is clear that the limiting factor in accuracy at large $x$ is the computation of numpy's inverse normal distribution, 'norm.ppf', rather than the specifics of our iterative implementation or definition of convergence.